In [3]:
import numpy as np
import cPickle
from collections import OrderedDict
import collections
from nltk.metrics import *

In [4]:

# load the data molde
input_handle = open('data/bg_session.ctx_ADJ.mdl', 'r')

# load the tuple dict and the query dict
tuple_dict = cPickle.load(input_handle)
query_to_id = cPickle.load(input_handle)

In [36]:
# make a inverted version of the query to id dict
id_to_query =  {v: k for k, v in query_to_id.iteritems()}

In [16]:
"""
When using enumerate you can only use this ones for the data set, you need to reload the data
before you can use emenumerate again
"""
def open_data():
    val_sessions = open('data/val_session.ctx', 'r')
    train_session = open('data/tr_session.ctx', 'r')
    return train_session, val_sessions

In [6]:
# use the keys (tuples with two query id's) of the tuple dict to make a new dict 
tuple_pairs = tuple_dict.keys()

In [7]:
search_dict = collections.defaultdict(dict)
"""
make a new dict with key anchor query, as value we have a new dict with keys previous query and 
their value count 

dict[anchor_query] = { previous_query: count_value}

"""



for _tuple in tuple_pairs:
    search_dict[_tuple[1]][_tuple[0]] = tuple_dict[_tuple] 

In [37]:
"""
Func to print the suggested query id's as strings using the id_to_query map
"""
def print_suggestion(suggestions):
    for suggest in suggestions:
        print id_to_query[suggest[0]]

In [55]:
"""
Function that makes suggestions for a session

Input: session file, *.ctx
Output: dict with key:session_idx value: (target_query,anchor_query, session, suggestions)

"""

def make_suggestions(session_file, recent_queries=10,num_suggestions=20):
    # make a dict to save all the results
    suggestion_dict = {}
    
    # loop over every session in the *.ctx file
    for idx, line in enumerate(session_file):
        # queries are tab-separated 
        session = line.strip().split('\t')
        
        # we need 10 context queries and 1 target query, min session lengt has to be 11
        if len(session) >= recent_queries+1:
            target_query = session[-1] # target query is the last query Qm
            anchor_query = session[-2] # Anchor query is the query Qm-1
            context = session[:-1] # Qm-1 till Q1 are the context queries
            
            # find anchor in the background set
            if anchor_query in query_to_id:
                key =  query_to_id[anchor_query] # the key of the query in the bg-set 
                # check if target query and anchor query are in the background set
                if key in search_dict and target_query in query_to_id:
                    """
                    We could use the search dict to find all the queries that follow the anchor query 
                    in the bg set, we use this queries as suggestions
                    """
                    suggestions = search_dict[key]
                    if len(suggestions) > num_suggestions: # we need at least 20 suggestions 
                        target_key = query_to_id[target_query] # find the key of the target query
                        
                        if target_key in suggestions: # check if target query is among the suggestions 
                            # we have a valid session, now we list all the suggestions and sort them
                            list_suggestions = [(key, suggestions[key] )for key in suggestions.keys()]
                            sorted_suggestions = sorted(list_suggestions, key=lambda x: x[1])[::-1]
                            #take only the top 20 suggestions based on counts 
                            suggestions = sorted_suggestions[0:num_suggestions]
                            # save this in the dict key(idx):(target_query,anchor_query, session, suggestions)
                            suggestion_dict[idx] = (target_query,anchor_query, session, suggestions)
    return suggestion_dict

In [56]:
train_session, val_sessions = open_data() # reload the data

# dicts with results
suggestion_train = make_suggestions(train_session)
suggestion_val = make_suggestions(val_sessions)

In [ ]:
"""
Function that returens a feature vector for every suggestion 

Input: suggestion_dict
Output: per session a matrix [17,20] with the feature vectors 
"""

def make_suggestion_features(suggestion_dict, num_features=17):
    for session_key in suggestion_dict.keys():
        session_tuple = suggestion_dict[session_key]
        target_query = session_tuple[0]
        anchor_query = session_tuple[1]
        suggestions = suggestions[3]
        for idx, suggestion in enumerate(suggestions):
            suggestion_id = suggestion[0]
             = id_to_query[suggestion_id]
            """"
            For each candidate suggestion, we count how many times it follows 
            the anchor query in the background data and add this count as a feature.
            """
            follow_anchor_count = suggestion[1]

            """
            Additionally, we use the frequency of the anchor query in the background data.
            """
            bg_freq = None 

            """
            We also add the Levenshtein distance between the anchor and the suggestion.
            """
            levenshtein_distance = edit_distance(anchor_query, query_string)

            """
            The suggestion length (characters and words)
            """
            chars_leng = None # met of zonder spaties?
            word_leng = None
 